In [1]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset, DataLoader
from skimage import io

import pandas as pd
import numpy as np
import os
import time
from tempfile import TemporaryDirectory

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from tqdm import tqdm

In [2]:
data_dir = 'dataset'

In [11]:
class CartPoleDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.df.iloc[idx, 0])
        image = io.imread(img_name)/255
        image = np.moveaxis(image, -1, 0)
        image = torch.tensor(image, dtype=torch.float32)
        labels = self.df.iloc[idx, 1:]
        labels = np.array(labels, dtype=np.float32)
        if self.transform:
            image = self.transform(image)
        sample = {'image': image, 'label': labels}


        return sample

In [12]:
train_ds = CartPoleDataset('dataset/train.csv', data_dir, torchvision.transforms.Resize(224))
test_ds = CartPoleDataset('dataset/test.csv', data_dir, torchvision.transforms.Resize(224))

In [13]:
dataloaders = {"train":DataLoader(train_ds, batch_size=32), "val":DataLoader(test_ds, batch_size=32)}
dataset_sizes = {"train": len(train_ds), "val": len(test_ds)}

In [32]:
def train_model(model, loss_fn, optimizer, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')
        model_params_path = os.path.join("models", 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        torch.save(model.state_dict(), model_params_path)
        best_loss = 0.0002822384874802083

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0

                # Iterate over data.
                for d in tqdm(dataloaders[phase]):
                    inputs = d['image']
                    labels = d['label']
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        loss = loss_fn(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)

                epoch_loss = running_loss / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss}')

                # deep copy the model
                if phase == 'val' and epoch_loss < best_loss:
                    best_loss = epoch_loss
                    torch.save(model.state_dict(), best_model_params_path)
                    torch.save(model.state_dict(), model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Loss: {best_loss:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [15]:
model_ft = torchvision.models.resnet18(weights='IMAGENET1K_V1')

In [16]:
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

In [17]:
model_ft = model_ft.to(device)

In [18]:
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model_ft.parameters(), lr=3e-4)

In [19]:
model_ft = train_model(model_ft, loss, optimizer,
                       num_epochs=25)

Epoch 0/24
----------


  0%|                                                                                                      | 0/625 [00:00<?, ?it/s]/home/aparna/whisper_transcription/env/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|████████████████████████████████████████████████████████████████████████████████████████████| 625/625 [03:20<00:00,  3.12i

train Loss: 0.0147


100%|████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:45<00:00,  3.47it/s]


val Loss: 0.0071

Epoch 1/24
----------


...

train Loss: 0.0010


100%|████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:44<00:00,  3.49it/s]


val Loss: 0.0024

Epoch 6/24
----------


  7%|██████▋                                                                                      | 45/625 [00:14<03:06,  3.12it/s]


KeyboardInterrupt: 

In [20]:
model_ft.load_state_dict(torch.load(os.path.join("models", 'best_model_params.pt')))

<All keys matched successfully>

In [28]:
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model_ft.parameters(), lr=5e-5)

In [29]:
model_ft = train_model(model_ft, loss, optimizer,
                       num_epochs=25)

Epoch 0/24
----------


100%|████████████████████████████████████████████████████████████████████████████████████████████| 625/625 [03:15<00:00,  3.20it/s]


train Loss: 0.0005111313453409821


100%|████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:44<00:00,  3.52it/s]


val Loss: 0.0002822384874802083

Epoch 1/24
----------


...

train Loss: 0.00029351560574723405


100%|████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:44<00:00,  3.54it/s]


val Loss: 0.0002966131589375436

Epoch 3/24
----------


  4%|████                                                                                         | 27/625 [00:08<03:10,  3.15it/s]


KeyboardInterrupt: 

In [30]:
model_ft.load_state_dict(torch.load(os.path.join("models", 'best_model_params.pt')))

<All keys matched successfully>

In [31]:
loss = nn.MSELoss()
optimizer = torch.optim.Adam(model_ft.parameters(), lr=1e-5)

In [33]:
model_ft = train_model(model_ft, loss, optimizer,
                       num_epochs=25)

Epoch 0/24
----------


100%|████████████████████████████████████████████████████████████████████████████████████████████| 625/625 [03:14<00:00,  3.21it/s]


train Loss: 0.0001996183784562163


100%|████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:44<00:00,  3.52it/s]


val Loss: 0.00015371240702224896

Epoch 1/24
----------


...

train Loss: 0.00011568377406802029


100%|████████████████████████████████████████████████████████████████████████████████████████████| 157/157 [00:44<00:00,  3.50it/s]


val Loss: 0.00014157925606705248

Epoch 7/24
----------


 28%|█████████████████████████▍                                                                  | 173/625 [00:54<02:23,  3.15it/s]


KeyboardInterrupt: 